# Simple RAG (Retrieval-Augmented Generation) Implementation with LangChain 1.0+

## Overview
This notebook demonstrates a complete RAG pipeline using **LangChain 1.0+ with LCEL** (LangChain Expression Language).

### What is RAG?
RAG combines retrieval of relevant documents with generation from a Large Language Model (LLM):
1. **Retrieval**: Find relevant information from a knowledge base
2. **Augmentation**: Add retrieved context to the prompt
3. **Generation**: LLM generates answers based on the context

### Pipeline Flow:
```
PDF Documents → Load → Split into Chunks → Create Embeddings → Store in Vector DB
                                                                         ↓
User Query → Retrieve Similar Chunks → Combine with Query → LLM → Answer
```

### Components Used:
- **Document Loader**: PyPDFLoader (for PDF processing)
- **Text Splitter**: RecursiveCharacterTextSplitter (smart chunking)
- **Embeddings**: OpenAI text-embedding-3-small (vector representations)
- **Vector Store**: FAISS (fast similarity search)
- **LLM**: OpenAI GPT-4-Turbo or GPT-3.5-Turbo
- **Chain Builder**: LCEL (LangChain Expression Language)

### LangChain 1.0+ Features:
- ✅ Modern LCEL syntax with pipe operator `|`
- ✅ More readable and composable chains
- ✅ Better streaming support
- ✅ Type-safe operations

---

## 1. Installation & Setup

First, install all required packages. Make sure you have Python 3.9+ installed.

In [ ]:
# Install required packages
# Uncomment and run ONE of the following options:

# Option 1: Install from requirements.txt (RECOMMENDED)
# !pip install -r requirements.txt

# Option 2: Install all packages individually
# !pip install langchain langchain-core langchain-openai langchain-community langchain-text-splitters faiss-cpu pypdf python-dotenv tiktoken jupyter notebook

# Option 3: Quick install (if you're having import issues)
# !pip install --upgrade langchain langchain-core langchain-openai langchain-community langchain-text-splitters

## 2. Import Required Libraries

Import all necessary modules with explanations of what each does.

### Verify Installation

If you encounter import errors, run this cell first to check package versions:

In [ ]:
%pip install langchain langchain-core langchain-openai langchain-community langchain-text-splitters faiss-cpu pypdf python-dotenv tiktoken

In [7]:
# Check installed package versions
import sys
from importlib.metadata import version

try:
    import langchain
    print(f"✓ langchain: {langchain.__version__}")
except:
    print("✗ langchain not installed")

try:
    import langchain_core
    print(f"✓ langchain-core: {langchain_core.__version__}")
except:
    print("✗ langchain-core not installed - REQUIRED!")
    print("  Run: pip install langchain-core")

try:
    import langchain_openai
    print(f"✓ langchain-openai: {version('langchain-openai')}")
except:
    print("✗ langchain-openai not installed")

try:
    import langchain_community
    print(f"✓ langchain-community: {langchain_community.__version__}")
except:
    print("✗ langchain-community not installed")

print(f"\nPython version: {sys.version}")
print("\nIf any packages are missing, run:")
print("pip install langchain langchain-core langchain-openai langchain-community langchain-text-splitters faiss-cpu pypdf python-dotenv tiktoken")

✓ langchain: 1.1.0
✓ langchain-core: 1.1.0
✓ langchain-openai: 1.1.0
✓ langchain-community: 0.4.1

Python version: 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 16:37:03) [MSC v.1929 64 bit (AMD64)]

If any packages are missing, run:
pip install langchain langchain-core langchain-openai langchain-community langchain-text-splitters faiss-cpu pypdf python-dotenv tiktoken


In [8]:
# Standard library imports
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
from pathlib import Path

# Environment variable management - for secure API key handling
from dotenv import load_dotenv

# LangChain Document Loaders - for loading PDF documents
from langchain_community.document_loaders import PyPDFLoader

# LangChain Text Splitters - for breaking documents into manageable chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

# OpenAI Integration - for embeddings and LLM
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

# Vector Store - FAISS for efficient similarity search
from langchain_community.vectorstores import FAISS

# LangChain Core Components
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

print("✓ All imports successful!")
print("✓ Compatible with LangChain 1.0+")

✓ All imports successful!
✓ Compatible with LangChain 1.0+


## 3. Environment Configuration

### Setting up OpenAI API Key

You have two options:
1. **Recommended**: Create a `.env` file with `OPENAI_API_KEY=your_key_here`
2. **Alternative**: Set it directly in code (not recommended for production)

Get your API key from: https://platform.openai.com/api-keys

In [9]:
# Load environment variables from .env file
load_dotenv()

# Verify API key is loaded
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  WARNING: OPENAI_API_KEY not found!")
    print("Please set it in .env file or uncomment the line below:")
    # os.environ["OPENAI_API_KEY"] = "your_api_key_here"
else:
    api_key = os.getenv("OPENAI_API_KEY")
    print("✓ OpenAI API Key loaded successfully!")
    print(f"✓ Key starts with: {api_key[:8]}..." if api_key else "✓ API Key is set but could not be displayed.")

✓ OpenAI API Key loaded successfully!
✓ Key starts with: sk-svcac...


## 4. Document Loading

### Loading PDF Documents

PyPDFLoader extracts text from PDF files page by page. Each page becomes a separate document with metadata (page number, source file).

**How it works:**
- Reads PDF files and extracts text content
- Preserves page numbers for source tracking
- Returns Document objects with `.page_content` and `.metadata`

**Note**: Update the `pdf_path` variable to point to your PDF file(s).

In [10]:
# ===== CONFIGURATION: Update this path to your PDF file =====
pdf_path = ".\\pdfs\\attention.pdf"  # Change this to your PDF file path
# =============================================================

# Check if file exists
if not os.path.exists(pdf_path):
    print(f"⚠️  ERROR: File '{pdf_path}' not found!")
    print("Please update the pdf_path variable with your PDF file location.")
else:
    # Initialize the PDF loader
    loader = PyPDFLoader(pdf_path)
    
    # Load all pages from the PDF
    # Each page becomes a separate Document object
    documents = loader.load()
    
    # Display information about loaded documents
    print(f"✓ Loaded {len(documents)} pages from '{pdf_path}'")
    print(f"\n--- First Document Preview ---")
    print(f"Content (first 500 chars): {documents[0].page_content[:500]}...")
    print(f"\nMetadata: {documents[0].metadata}")
    print(f"\nTotal characters across all pages: {sum(len(doc.page_content) for doc in documents):,}")

✓ Loaded 15 pages from '.\pdfs\attention.pdf'

--- First Document Preview ---
Content (first 500 chars): Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.edu
Łukasz ...

Metadata: {'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '.\\pdfs\\attention.pdf', 'total

### Loading Multiple PDFs (Optional)

If you have multiple PDF files, you can load them all at once:

In [11]:
# Example: Loading multiple PDFs from a directory
# Uncomment and modify if you want to load multiple files

pdf_directory = "./pdfs"  # Directory containing your PDFs
all_documents = []

if os.path.exists(pdf_directory):
    pdf_files = list(Path(pdf_directory).glob("*.pdf"))
    print(f"Found {len(pdf_files)} PDF files")
    
    for pdf_file in pdf_files:
        loader = PyPDFLoader(str(pdf_file))
        docs = loader.load()
        all_documents.extend(docs)
        print(f"  ✓ Loaded {len(docs)} pages from {pdf_file.name}")
    
    print(f"\nTotal pages loaded: {len(all_documents)}")
    documents = all_documents  # Use this for the rest of the pipeline

Found 3 PDF files
  ✓ Loaded 15 pages from attention.pdf
  ✓ Loaded 15 pages from attention.pdf
  ✓ Loaded 19 pages from rag.pdf
  ✓ Loaded 19 pages from rag.pdf
  ✓ Loaded 21 pages from ragsurvey.pdf

Total pages loaded: 55
  ✓ Loaded 21 pages from ragsurvey.pdf

Total pages loaded: 55


## 5. Text Splitting

### Why Split Documents?
- LLMs have token limits (e.g., 4K, 8K, 128K tokens)
- Smaller chunks = more precise retrieval
- Balance: chunks must be large enough to contain meaningful context but small enough to be specific

### RecursiveCharacterTextSplitter
This splitter tries to keep related text together by recursively splitting on:
1. Paragraphs (`\n\n`)
2. Lines (`\n`)
3. Sentences (`. `)
4. Words (` `)
5. Characters (as last resort)

**Parameters:**
- `chunk_size=1024`: Target size for each chunk (in characters)
- `chunk_overlap=128`: Overlap between chunks to maintain context continuity
- Overlap prevents important information from being split across chunks

In [12]:
# Initialize the text splitter with recommended settings
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,        # Maximum characters per chunk (roughly 200-250 tokens)
    chunk_overlap=128,      # Characters overlap between chunks (maintains context)
    length_function=len,    # Function to measure chunk length
    separators=["\n\n", "\n", " ", ""]  # Try to split on paragraphs first, then lines, etc.
)

# Split the documents into chunks
# This creates smaller, manageable pieces while preserving semantic meaning
chunks = text_splitter.split_documents(documents)

# Display splitting results
print(f"✓ Split {len(documents)} documents into {len(chunks)} chunks")
print(f"\nAverage chunk size: {sum(len(chunk.page_content) for chunk in chunks) / len(chunks):.0f} characters")

# Preview a few chunks
print(f"\n--- Chunk Examples ---")
for i, chunk in enumerate(chunks[:3]):
    print(f"\nChunk {i+1} (length: {len(chunk.page_content)} chars):")
    print(f"{chunk.page_content[:200]}...")
    print(f"Metadata: {chunk.metadata}")

✓ Split 55 documents into 267 chunks

Average chunk size: 898 characters

--- Chunk Examples ---

Chunk 1 (length: 986 chars):
Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
...
Metadata: {'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'pdfs\\attention.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}

Chunk 2 (length: 944 chars):
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more pa...
Metad

## 6. Creating Embeddings

### What are Embeddings?
Embeddings are vector representations of text that capture semantic meaning. Similar texts have similar vectors.

**Example**: 
- "dog" and "puppy" → similar vectors (close in vector space)
- "dog" and "spaceship" → different vectors (far apart)

### OpenAI text-embedding-3-small
- **Dimensions**: 1536 (each text becomes a 1536-dimensional vector)
- **Cost**: $0.00002 per 1,000 tokens (very affordable)
- **Performance**: 62.3% on MTEB benchmark
- **Speed**: Fast and efficient

**Alternative**: `text-embedding-3-large` for higher quality (64.6% MTEB) at higher cost

In [13]:
# Initialize OpenAI Embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",  # Latest, cost-effective embedding model
    # dimensions=1536
    # Alternative: "text-embedding-3-large" for better quality
)

# Test the embeddings with a sample text
sample_text = "This is a test sentence to demonstrate embeddings."
sample_embedding = embeddings.embed_query(sample_text)

print(f"✓ Embeddings model initialized: text-embedding-3-small")
print(f"✓ Embedding dimension: {len(sample_embedding)}")
print(f"✓ Sample embedding (first 10 values): {sample_embedding[:10]}")
print(f"\nℹ️  Each chunk will be converted to a {len(sample_embedding)}-dimensional vector for similarity search")

✓ Embeddings model initialized: text-embedding-3-small
✓ Embedding dimension: 1536
✓ Sample embedding (first 10 values): [0.020370882004499435, -0.0031641265377402306, -0.0005454652709886432, 0.0045827641151845455, -0.015004359185695648, -0.034060992300510406, 0.0176328606903553, 0.01959054544568062, 0.0013125392142683268, 0.00596546521410346]

ℹ️  Each chunk will be converted to a 1536-dimensional vector for similarity search


### Google Gemini Embeddings (Optional)

In [14]:
# Below is the code for Gemini Embeddings:

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

# # Test the embeddings with a sample text
sample_text = "This is a test sentence to demonstrate embeddings."

# Control your dimensionality by the parameter herein below.. 

sample_embedding = embeddings.embed_query(sample_text,output_dimensionality=1536)

print(f"✓ Embeddings model initialized: gemini-embedding-001")
print(f"✓ Embedding dimension: {len(sample_embedding)}")
print(f"✓ Sample embedding (first 10 values): {sample_embedding[:10]}")
print(f"✓ Each chunk will be converted to a {len(sample_embedding)}-dimensional vector for similarity search")

✓ Embeddings model initialized: gemini-embedding-001
✓ Embedding dimension: 1536
✓ Sample embedding (first 10 values): [-0.029574524611234665, 0.022307422012090683, 0.006308525335043669, -0.08331369608640671, -0.0002317591424798593, -0.003521788865327835, 0.006556599400937557, 0.011158722452819347, 0.008888664655387402, -0.009683165699243546]
✓ Each chunk will be converted to a 1536-dimensional vector for similarity search


## 7. Creating Vector Store (FAISS)

### What is a Vector Store?
A vector store (or vector database) stores embeddings and enables fast similarity search.

### FAISS (Facebook AI Similarity Search)
- **Fast**: Optimized for billion-scale vector search
- **Local**: Runs on your machine, no cloud dependency
- **Efficient**: Uses advanced indexing algorithms

### How Similarity Search Works:
1. Convert query to embedding vector
2. Find vectors in the database most similar to query vector (using cosine similarity or Euclidean distance)
3. Return the corresponding text chunks

**This cell will:**
1. Convert all chunks to embeddings (may take a minute for large documents)
2. Build a FAISS index
3. Save to disk for future use

### FAISS-CPU Version Compatibility Issue

If you encounter issues with `faiss-cpu` installation, try:

```bash
uv pip uninstall faiss-cpu
uv pip install faiss-cpu==1.12.0
```

Or for conda users:
```bash
conda install -c conda-forge faiss-cpu==1.12.0
```

In [15]:
!uv pip show faiss-cpu

Name: faiss-cpu
Version: 1.12.0
Location: d:\AiCode\LcRAGBtCmp\simple-rag-langchain\lcrag_venv\Lib\site-packages
Requires: numpy, packaging
Required-by:


Using Python 3.13.5 environment at: lcrag_venv


In [18]:
# Create FAISS vector store from document chunks
# This step converts each chunk to an embedding and stores it
print(f"Billing Set-up to the following Account - Capital One Watch !")
print(f"KrshNaik-AgentiAI-Gogle-Gemini  	gen-lang-client-0899462979")
print(f"Creating FAISS index from {len(chunks)} chunks...")
print("This may take a minute depending on the number of chunks...")

#Billing Set-up to the following Account - Capital One Watch !
#KrshNaik-AgentiAI-Gogle-Gemini  	gen-lang-client-0899462979
#Creating FAISS index from 267 chunks...
#This may take a minute depending on the number of chunks...
#✓ FAISS vector store created successfully!
#✓ Indexed 267 document chunks
#✓ Vector store saved to './faiss_index'

#ℹ️  You can reload this index later using: FAISS.load_local('./faiss_index', embeddings)

vectorstore = FAISS.from_documents(
    documents=chunks,      # Our split document chunks
    embedding=embeddings   # OpenAI embedding model
)

print(f"✓ FAISS vector store created successfully!")
print(f"✓ Indexed {len(chunks)} document chunks")

# Save the vector store to disk for later use
# This allows you to reload the index without re-processing documents
vectorstore_path = "./faiss_index"
vectorstore.save_local(vectorstore_path)
print(f"✓ Vector store saved to '{vectorstore_path}'")
print(f"\nℹ️  You can reload this index later using: FAISS.load_local('{vectorstore_path}', embeddings)")

Billing Set-up to the following Account - Capital One Watch !
KrshNaik-AgentiAI-Gogle-Gemini  	gen-lang-client-0899462979
Creating FAISS index from 267 chunks...
This may take a minute depending on the number of chunks...
✓ FAISS vector store created successfully!
✓ Indexed 267 document chunks
✓ Vector store saved to './faiss_index'

ℹ️  You can reload this index later using: FAISS.load_local('./faiss_index', embeddings)
✓ FAISS vector store created successfully!
✓ Indexed 267 document chunks
✓ Vector store saved to './faiss_index'

ℹ️  You can reload this index later using: FAISS.load_local('./faiss_index', embeddings)


### ChromaDB Vector Store (Optional)

In [19]:
#ChromaDB has better Python 3.13 support. Replace cells 21-24 with:

# Instead of FAISS, use ChromaDB
#Billing Set-up to the following Account - Capital One Watch !
#KrshNaik-AgentiAI-Gogle-Gemini  	gen-lang-client-0899462979

from langchain_community.vectorstores import Chroma

# # Create ChromaDB vector store
print(f"Creating ChromaDB from {len(chunks)} chunks...")
#vectorstore = Chroma.from_documents(
#     documents=chunks,
#     embedding=embeddings,
#     persist_directory="./chroma_db"
#)
print("✓ ChromaDB vector store created!")

#Creating ChromaDB from 267 chunks...
#✓ ChromaDB vector store created!


Creating ChromaDB from 267 chunks...
✓ ChromaDB vector store created!


### Loading a Saved Vector Store (Optional)

If you've already created a vector store, you can load it instead of recreating:

In [21]:
#Uncomment to load an existing vector store instead of creating a new one
vectorstore_path = "./faiss_index"
vectorstore = FAISS.load_local(
    vectorstore_path, 
    embeddings,
    allow_dangerous_deserialization=True  # Required for loading pickled data
 )
print(f"✓ Loaded existing vector store from '{vectorstore_path}'")

# vectorstore_path = "./faiss_index" --- IGNORE ---
# vectorstore = FAISS.load_local( --- IGNORE ---
#     vectorstore_path,  --- IGNORE ---
#     embeddings, --- IGNORE ---
#     allow_dangerous_deserialization=True  # Required for loading pickled data --- IGNORE ---
#  ) --- IGNORE ---
# print(f"✓ Loaded existing vector store from '{vectorstore_path}'") --- IGNORE

✓ Loaded existing vector store from './faiss_index'


In [22]:
# Create a retriever from the vector store
retriever = vectorstore.as_retriever(
    search_type="similarity",    # Use cosine similarity for search
    search_kwargs={"k": 4}        # Retrieve top 4 most relevant chunks
)

print("✓ Retriever configured successfully")
print(f"  - Search type: similarity")
print(f"  - Number of documents to retrieve (k): 4")

# Test the retriever with a sample query
# Note: In LangChain 1.0+, use .invoke() instead of .get_relevant_documents()
test_query = "What is the main topic of this document?"
retrieved_docs = retriever.invoke(test_query)  # LangChain 1.0+ method

print(f"\n--- Retriever Test ---")
print(f"Query: '{test_query}'")
print(f"Retrieved {len(retrieved_docs)} documents:")

for i, doc in enumerate(retrieved_docs):
    print(f"\nDocument {i+1}:")
    print(f"  Content preview: {doc.page_content[:150]}...")
    print(f"  Metadata: {doc.metadata}")

✓ Retriever configured successfully
  - Search type: similarity
  - Number of documents to retrieve (k): 4

--- Retriever Test ---
Query: 'What is the main topic of this document?'
Retrieved 4 documents:

Document 1:
  Content preview: 1
Retrieval-Augmented Generation for Large
Language Models: A Survey
Yunfan Gaoa, Yun Xiongb, Xinyu Gao b, Kangxiang Jia b, Jinliu Pan b, Yuxi Bic, Yi...
  Metadata: {'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-03-28T00:54:45+00:00', 'author': '', 'keywords': '', 'moddate': '2024-03-28T00:54:45+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'pdfs\\ragsurvey.pdf', 'total_pages': 21, 'page': 0, 'page_label': '1'}

Document 2:
  Content preview: for continuous knowledge updates and integration of domain-
specific information. RAG synergistically merges LLMs’ intrin-
sic knowledge with the 

## 9. Configuring the Language Model (LLM)

### LLM Selection
The LLM generates the final answer based on retrieved context.

### Available Models:
1. **gpt-4-turbo-2025-04-09**: Most capable, best quality, slower, more expensive
2. **gpt-4o**: Fast GPT-4 level performance, good balance
3. **gpt-3.5-turbo**: Fast and cheap, good for simpler queries

### Temperature:
- **0**: Deterministic, focused answers (recommended for factual Q&A)
- **0.7**: More creative, varied responses
- **1.0**: Most creative, less predictable

### Max Tokens:
Controls the maximum length of the generated response.

In [ ]:
# Initialize the ChatOpenAI model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
      model="gpt-4o-mini",  # "gpt-4-turbo-2024-04-09",  # Choose your model
      # Alternative options:
      # model="gpt-4o-mini",           # Faster GPT-4 performance, good 
      # balance,
      # model="gpt-3.5-turbo",    # Faster and cheaper option

      temperature=0         # 0 = deterministic, factual responses (recommended for Q&A)
      # max_tokens is not a valid parameter here
  )

print("✓ LLM configured successfully")
print(f"  - Model: gpt-4o-mini")
print(f"  - Temperature: 0 (deterministic)")
print(f"  - Max tokens: 2000")

# Test the LLM with a simple query
test_response = llm.invoke("Say 'Hello, I am ready to answer questions!'")
print(f"\nLLM Test Response: {test_response.content}")

#   📝 Explanation of Parameters:

#   Model Selection:

#   # Option 1: Best quality (slower, more expensive)
#   llm = ChatOpenAI(model="gpt-4-turbo-2024-04-09")

#   # Option 2: Fast GPT-4 performance (balanced)
#   llm = ChatOpenAI(model="gpt-4o")

#   # Option 3: Fast and cheap (good for testing)
#   llm = ChatOpenAI(model="gpt-3.5-turbo")

#   Temperature:

#   temperature=0    # Deterministic, focused (best for factual Q&A)
#   temperature=0.7  # More creative, varied responses
#   temperature=1.0  # Most creative, less predictable

#   Max Tokens:

#   max_tokens=2000  # Controls maximum response length

✓ LLM configured successfully
  - Model: gpt-4o-mini
  - Temperature: 0 (deterministic)
  - Max tokens: 2000

LLM Test Response: Hello, I am ready to answer questions!

LLM Test Response: Hello, I am ready to answer questions!


## 10. Creating the RAG Chain (LangChain 1.0+ LCEL)

### What is a RAG Chain?
The RAG chain combines retrieval and generation into a single workflow:
1. User asks a question
2. Retriever finds relevant documents
3. Documents are formatted as context
4. LLM generates answer using the context

### LangChain 1.0+ LCEL (LangChain Expression Language)
LangChain 1.0+ uses LCEL, a declarative way to build chains using the pipe operator `|`.

**Benefits:**
- More intuitive and readable
- Better streaming support
- Easier to debug and modify
- Type-safe and composable

**Components:**
- **RunnablePassthrough**: Passes input through unchanged
- **Pipe operator (|)**: Chains components together
- **StrOutputParser**: Converts LLM output to string

In [24]:
#Step 1: Create the Retriever 
# Create a retriever from the vector store
retriever = vectorstore.as_retriever(
      search_type="similarity",    # Use cosine similarity for search
      search_kwargs={"k": 4}        # Retrieve top 4 most relevant chunks
  )

print("✓ Retriever configured successfully")
print(f"  - Search type: similarity")
print(f"  - Number of documents to retrieve (k): 4")

  # Test the retriever with a sample query
test_query = "What is the main topic of this document?"
retrieved_docs = retriever.invoke(test_query)

print(f"\n--- Retriever Test ---")
print(f"Query: '{test_query}'")
print(f"Retrieved {len(retrieved_docs)} documents:")

for i, doc in enumerate(retrieved_docs):
    print(f"\nDocument {i+1}:")
    print(f"  Content preview: {doc.page_content[:150]}...")
    print(f"  Metadata: {doc.metadata}")

✓ Retriever configured successfully
  - Search type: similarity
  - Number of documents to retrieve (k): 4

--- Retriever Test ---
Query: 'What is the main topic of this document?'
Retrieved 4 documents:

Document 1:
  Content preview: 1
Retrieval-Augmented Generation for Large
Language Models: A Survey
Yunfan Gaoa, Yun Xiongb, Xinyu Gao b, Kangxiang Jia b, Jinliu Pan b, Yuxi Bic, Yi...
  Metadata: {'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-03-28T00:54:45+00:00', 'author': '', 'keywords': '', 'moddate': '2024-03-28T00:54:45+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'pdfs\\ragsurvey.pdf', 'total_pages': 21, 'page': 0, 'page_label': '1'}

Document 2:
  Content preview: for continuous knowledge updates and integration of domain-
specific information. RAG synergistically merges LLMs’ intrin-
sic knowledge with the 

In [25]:
# Define the prompt template for the RAG system
# This tells the LLM how to use the retrieved context
system_prompt = (
    "You are a helpful assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer based on the context, say that you don't know. "
    "Keep the answer concise and accurate.\n\n"
    "Context: {context}\n\n"
    "Question: {question}"
)

# Create the prompt template
prompt = ChatPromptTemplate.from_template(system_prompt)

# Helper function to format documents
def format_docs(docs):
    """Format retrieved documents into a single string."""
    return "\n\n".join(doc.page_content for doc in docs)

# Build the RAG chain using LangChain 1.0+ LCEL (LangChain Expression Language)
# This uses the pipe operator (|) to chain components together
rag_chain = (
    {
        "context": retriever | format_docs,  # Retrieve docs and format them
        "question": RunnablePassthrough()      # Pass through the question
    }
    | prompt           # Format with prompt template
    | llm              # Generate answer with LLM
    | StrOutputParser() # Parse output to string
)

print("✓ RAG chain created successfully using LangChain 1.0+ LCEL!")
print("\nRAG Pipeline Flow:")
print("  1. User provides a query")
print("  2. Retriever finds top 4 relevant chunks")
print("  3. Chunks are formatted as context")
print("  4. Context + question are formatted with prompt template")
print("  5. LLM generates answer based on context")
print("  6. Answer is parsed and returned to user")

✓ RAG chain created successfully using LangChain 1.0+ LCEL!

RAG Pipeline Flow:
  1. User provides a query
  2. Retriever finds top 4 relevant chunks
  3. Chunks are formatted as context
  4. Context + question are formatted with prompt template
  5. LLM generates answer based on context
  6. Answer is parsed and returned to user


In [27]:
# Example Query 1: General question about the document
query1 = "What is the main topic or subject of this document?"

print(f"Query: {query1}")
print("\nProcessing...\n")

# With LangChain 1.0+, we invoke the chain with the question directly
answer = rag_chain.invoke(query1)

print("=" * 80)
print("ANSWER:")
print("=" * 80)
print(answer)
print("\n" + "=" * 80)

# To see which documents were retrieved, we can call the retriever separately
print("\nSOURCE DOCUMENTS USED:")
print("=" * 80)
retrieved_docs = retriever.invoke(query1)
for i, doc in enumerate(retrieved_docs):
    print(f"\nDocument {i+1}:")
    print(f"  Source: {doc.metadata}")
    print(f"  Content: {doc.page_content[:200]}...")
    print("-" * 80)

Query: What is the main topic or subject of this document?

Processing...

ANSWER:
The main topic of the document is Retrieval-Augmented Generation (RAG) for knowledge-intensive natural language processing tasks, focusing on its implementation details, evaluation methods, and performance in various applications.


SOURCE DOCUMENTS USED:

Document 1:
  Source: {'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '2021-04-13T00:48:38+00:00', 'author': '', 'keywords': '', 'moddate': '2021-04-13T00:48:38+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'pdfs\\rag.pdf', 'total_pages': 19, 'page': 16, 'page_label': '17'}
  Content: Appendices for Retrieval-Augmented Generation for
Knowledge-Intensive NLP Tasks
A Implementation Details
For Open-domain QA we report test numbers using 15 retrieved documents for RAG-Token models.
Fo...
--------

In [28]:
# Example Query 2: Specific information extraction
query2 = "Can you summarize the key points from this document?"

print(f"Query: {query2}")
print("\nProcessing...\n")

answer = rag_chain.invoke(query2)

print("=" * 80)
print("ANSWER:")
print("=" * 80)
print(answer)
print("\n" + "=" * 80)

Query: Can you summarize the key points from this document?

Processing...

ANSWER:
The document discusses the challenges and methodologies related to Retrieval-Augmented Generation (RAG) systems. Key points include:

1. **Query Optimization**: Emphasizes the importance of formulating clear queries, as vague or complex questions can lead to ineffective retrieval. It highlights issues with language complexity and ambiguity.

2. **Query Expansion**: Suggests expanding a single query into multiple queries to enhance retrieval effectiveness.

3. **Reranking**: Describes the process of reordering document chunks to prioritize the most relevant results, using both rule-based and model-based methods.

4. **Context Selection/Compression**: Addresses misconceptions about retrieving and concatenating documents, advocating for a streamlined approach that reduces redundancy and noise while ensuring relevance.

5. **Task Adaptation**: Discusses the Task Adapter module, which customizes RAG for vari

### Please you Execute

In [29]:
# Example Query 3: Your custom question
# Replace this with your own question!
custom_query = "What specific details are mentioned about attention mechanisms?"

print(f"Query: {custom_query}")
print("\nProcessing...\n")

answer = rag_chain.invoke(custom_query)

print("=" * 80)
print("ANSWER:")
print("=" * 80)
print(answer)
print("\n" + "=" * 80)

Query: What specific details are mentioned about attention mechanisms?

Processing...

ANSWER:
The context mentions several specific details about attention mechanisms:

1. **Encoder-Decoder Attention**: In this mechanism, queries come from the previous decoder layer, while memory keys and values come from the encoder's output, allowing the decoder to attend to all positions in the input sequence.

2. **Self-Attention in Encoder**: In self-attention layers of the encoder, all keys, values, and queries come from the same source, allowing each position to attend to all positions in the previous encoder layer.

3. **Self-Attention in Decoder**: The decoder's self-attention layers allow each position to attend to all previous positions, preventing leftward information flow to maintain the auto-regressive property.

4. **Multi-Head Attention**: The model employs multi-head attention, which allows it to attend to information from different representation subspaces at different positions. It 

In [30]:
# Example Query 3: Your custom question
# Replace this with your own question!
custom_query = "What are the applications of Attention Mechanism?"

print(f"Query: {custom_query}")
print("\nProcessing...\n")

response3 = rag_chain.invoke(custom_query)

print("=" * 80)
print("ANSWER:")
print("=" * 80)
print(response3)
print("\n" + "=" * 80)

Query: What are the applications of Attention Mechanism?

Processing...

ANSWER:
The attention mechanism is integral to sequence modeling and transduction models in various tasks, allowing for the modeling of dependencies without regard to their distance in the input or output sequences. It is commonly used in natural language processing tasks such as machine translation, text summarization, and other applications that require understanding relationships in sequential data.



### FAISS Kernel Crash ISSUE

Testing retrieval with query: 'What is the main topic of this document?'
OMP: Error #15: Initializing libomp.dylib, but found libomp.dylib already initialized.
OMP: Hint This means that multiple copies of the OpenMP runtime have been linked into the program. That is dangerous, since it can degrade performance or cause incorrect results. The best thing to do is to ensure that only a single OpenMP runtime is linked into the process, e.g. by avoiding static linking of the OpenMP runtime in any library. As an unsafe, unsupported, undocumented workaround you can set the environment variable KMP_DUPLICATE_LIB_OK=TRUE to allow the program to continue to execute, but that may cause crashes or silently produce incorrect results. For more information, please see http://openmp.llvm.org/